# Constitution d'une base adresse à partir de fichiers DVF
- Adresse complète
- Code commune INSEE
- Code IRIS (code quartier de l'INSEE)
- Coordonnées GPS de la commune
- Coorodnnées GPS de l'adresse

## Lecture du fichier DVF

In [1]:
import warnings
from pandas.core.common import SettingWithCopyWarning
warnings.simplefilter(action="ignore", category=SettingWithCopyWarning)

In [2]:
import pandas as pd
import numpy as np
from geopy.geocoders import Nominatim
import json
import requests
import time

In [3]:
# Renseigner ici la première ligne et la dernière ligne de la base adresses qui vont être utilisées
row_first = 100000
row_last = 100500

In [4]:
# Chargement du jeu de données
df = pd.read_csv('valeursfoncieres-2020.txt',sep="|",decimal=",")
df.head()

C:\Users\olivier\anaconda\envs\Geo\lib\site-packages\IPython\core\interactiveshell.py:3165: DtypeWarning: Columns (14,18,23,24,26,28,41) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


,Code service CH,Reference document,1 Articles CGI,2 Articles CGI,3 Articles CGI,4 Articles CGI,5 Articles CGI,No disposition,Date mutation,Nature mutation,...,Surface Carrez du 5eme lot,Nombre de lots,Code type local,Type local,Identifiant local,Surface reelle bati,Nombre pieces principales,Nature culture,Nature culture speciale,Surface terrain
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,07/01/2020,Vente,...,NaN,0,NaN,NaN,NaN,NaN,NaN,T,NaN,1061.0
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,02/01/2020,Vente,...,NaN,0,NaN,NaN,NaN,NaN,NaN,BT,NaN,85.0
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,02/01/2020,Vente,...,NaN,0,NaN,NaN,NaN,NaN,NaN,T,NaN,1115.0
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,02/01/2020,Vente,...,NaN,0,NaN,NaN,NaN,NaN,NaN,T,NaN,1940.0
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,02/01/2020,Vente,...,NaN,0,NaN,NaN,NaN,NaN,NaN,T,NaN,1148.0


In [5]:
print(df.columns)

Index(['Code service CH', 'Reference document', '1 Articles CGI',
       '2 Articles CGI', '3 Articles CGI', '4 Articles CGI', '5 Articles CGI',
       'No disposition', 'Date mutation', 'Nature mutation', 'Valeur fonciere',
       'No voie', 'B/T/Q', 'Type de voie', 'Code voie', 'Voie', 'Code postal',
       'Commune', 'Code departement', 'Code commune', 'Prefixe de section',
       'Section', 'No plan', 'No Volume', '1er lot',
       'Surface Carrez du 1er lot', '2eme lot', 'Surface Carrez du 2eme lot',
       '3eme lot', 'Surface Carrez du 3eme lot', '4eme lot',
       'Surface Carrez du 4eme lot', '5eme lot', 'Surface Carrez du 5eme lot',
       'Nombre de lots', 'Code type local', 'Type local', 'Identifiant local',
       'Surface reelle bati', 'Nombre pieces principales', 'Nature culture',
       'Nature culture speciale', 'Surface terrain'],
      dtype='object')


## Constitution d'une base adresse à partir du fichier DVF

In [6]:
# focus on the address datas
ad = df[['No voie', 'Type de voie', 'Voie', 'Code postal','Commune', 'Code departement', 'Code commune']]
ad.shape

(2459560, 7)

In [7]:
ad.head()

,No voie,Type de voie,Voie,Code postal,Commune,Code departement,Code commune
0,NaN,NaN,FORTUNAT,1250.0,CEYZERIAT,1,72
1,NaN,NaN,TERRES DES CINQ SAULES,1290.0,LAIZ,1,203
2,NaN,NaN,BOIS DU CHAMP RION,1290.0,LAIZ,1,203
3,NaN,NaN,EN COROBERT,1290.0,LAIZ,1,203
4,NaN,NaN,TERRES DES CINQ SAULES,1290.0,LAIZ,1,203


In [8]:
# On supprime les adresses dupliquées pour éviter d'interroger les API plusieurs fois avec exactement la même adresse
ad = ad.drop_duplicates()
ad.shape

(1059450, 7)

In [9]:
# Traitement des codes postaux (transformés en INT sauf les NA)

ad['Code postal'] = ad['Code postal'].fillna(-1).astype(int).astype(str).replace('-1', np.nan)
ad.head()

,No voie,Type de voie,Voie,Code postal,Commune,Code departement,Code commune
0,NaN,NaN,FORTUNAT,1250,CEYZERIAT,1,72
1,NaN,NaN,TERRES DES CINQ SAULES,1290,LAIZ,1,203
2,NaN,NaN,BOIS DU CHAMP RION,1290,LAIZ,1,203
3,NaN,NaN,EN COROBERT,1290,LAIZ,1,203
6,NaN,RUE,DE LA CHARTREUSE,1960,PERONNAS,1,289


In [10]:
ad['Code departement'] = ad['Code departement'].apply(lambda x: str(x).zfill(2) if len(str(x)) == 1 else x)
ad['Code commune'] = ad['Code commune'].apply(lambda x: str(x).zfill(3) if len(str(x)) < 3 else x)
ad['Code commune INSEE'] = ad['Code departement'].astype(str) + ad['Code commune'].astype(str)
ad.head()

,No voie,Type de voie,Voie,Code postal,Commune,Code departement,Code commune,Code commune INSEE
0,NaN,NaN,FORTUNAT,1250,CEYZERIAT,01,072,01072
1,NaN,NaN,TERRES DES CINQ SAULES,1290,LAIZ,01,203,01203
2,NaN,NaN,BOIS DU CHAMP RION,1290,LAIZ,01,203,01203
3,NaN,NaN,EN COROBERT,1290,LAIZ,01,203,01203
6,NaN,RUE,DE LA CHARTREUSE,1960,PERONNAS,01,289,01289


In [11]:
# On formate les adresses
# tools functions : nan values management
def escapeNan(row):
    val = str(row)
    if val == "nan":
        val = " "
    else:
        val = val.replace('.0','') # if from float values
        val = val + " "
    return val

In [12]:
# Création d'un champ "Ville" pour interroger les coordonnées GPS lorsque le champ Adresse ne donne rien
ad['Ville'] = ad['Code postal'] + " " + ad['Commune']
ad.head()

,No voie,Type de voie,Voie,Code postal,Commune,Code departement,Code commune,Code commune INSEE,Ville
0,NaN,NaN,FORTUNAT,1250,CEYZERIAT,01,072,01072,1250 CEYZERIAT
1,NaN,NaN,TERRES DES CINQ SAULES,1290,LAIZ,01,203,01203,1290 LAIZ
2,NaN,NaN,BOIS DU CHAMP RION,1290,LAIZ,01,203,01203,1290 LAIZ
3,NaN,NaN,EN COROBERT,1290,LAIZ,01,203,01203,1290 LAIZ
6,NaN,RUE,DE LA CHARTREUSE,1960,PERONNAS,01,289,01289,1960 PERONNAS


In [13]:
# Modification des communes avec arrondissements
ad['Commune'] = ad['Commune'].replace(['MARSEILLE 1ER', 'MARSEILLE 2EME', 'MARSEILLE 3EME', 'MARSEILLE 4EME',
                                           'MARSEILLE 5EME', 'MARSEILLE 6EME', 'MARSEILLE 7EME', 'MARSEILLE 8EME',
                                           'MARSEILLE 9EME', 'MARSEILLE 10EME', 'MARSEILLE 11EME', 'MARSEILLE 12EME',
                                           'MARSEILLE 13EME', 'MARSEILLE 14EME', 'MARSEILLE 15EME', 'MARSEILLE 16EME',
                                           'LYON 1ER', 'LYON 2EME', 'LYON 3EME', 'LYON 4EME', 'LYON 5EME',
                                           'LYON 6EME', 'LYON 7EME', 'LYON 8EME', 'LYON 9EME',
                                           'PARIS 01', 'PARIS 02', 'PARIS 03', 'PARIS 04', 'PARIS 05', 'PARIS 06',
                                           'PARIS 07', 'PARIS 08', 'PARIS 09', 'PARIS 10', 'PARIS 11', 'PARIS 12',
                                           'PARIS 13', 'PARIS 14', 'PARIS 15', 'PARIS 16', 'PARIS 17', 'PARIS 18',
                                           'PARIS 19', 'PARIS 20'],
                                          ['MARSEILLE', 'MARSEILLE', 'MARSEILLE', 'MARSEILLE',
                                          'MARSEILLE', 'MARSEILLE', 'MARSEILLE', 'MARSEILLE',
                                          'MARSEILLE', 'MARSEILLE', 'MARSEILLE', 'MARSEILLE',
                                          'MARSEILLE', 'MARSEILLE', 'MARSEILLE', 'MARSEILLE',
                                          'LYON', 'LYON', 'LYON', 'LYON', 'LYON',
                                          'LYON', 'LYON', 'LYON', 'LYON',
                                          'PARIS', 'PARIS', 'PARIS', 'PARIS', 'PARIS', 'PARIS',
                                          'PARIS', 'PARIS', 'PARIS', 'PARIS', 'PARIS', 'PARIS',
                                          'PARIS', 'PARIS', 'PARIS', 'PARIS', 'PARIS', 'PARIS',
                                          'PARIS', 'PARIS'])

In [14]:
# Ajout d'apostrophes après L et D pour une meilleure reconnaissance des noms de rues
ad['Voie'] = ad['Voie'].replace([' L ', ' D '], [' L\'', ' D\''], regex = True)

In [15]:
# Création du champ "addressName" qui sera utilisée dans la requête API
ad['Adresse'] = ad.apply(lambda row: 
    escapeNan(row['No voie'])
    + escapeNan(row['Type de voie'])         
    + escapeNan(row['Voie'])
    + escapeNan(row['Code postal']).zfill(5) # pour respecter le format des codes postaux
    + str(row['Commune'])
    , axis = 1).str.strip()

In [16]:
ad.head()

,No voie,Type de voie,Voie,Code postal,Commune,Code departement,Code commune,Code commune INSEE,Ville,Adresse
0,NaN,NaN,FORTUNAT,1250,CEYZERIAT,01,072,01072,1250 CEYZERIAT,FORTUNAT 1250 CEYZERIAT
1,NaN,NaN,TERRES DES CINQ SAULES,1290,LAIZ,01,203,01203,1290 LAIZ,TERRES DES CINQ SAULES 1290 LAIZ
2,NaN,NaN,BOIS DU CHAMP RION,1290,LAIZ,01,203,01203,1290 LAIZ,BOIS DU CHAMP RION 1290 LAIZ
3,NaN,NaN,EN COROBERT,1290,LAIZ,01,203,01203,1290 LAIZ,EN COROBERT 1290 LAIZ
6,NaN,RUE,DE LA CHARTREUSE,1960,PERONNAS,01,289,01289,1960 PERONNAS,RUE DE LA CHARTREUSE 1960 PERONNAS


In [17]:
ad['Ville'] = ad['Ville'].replace('-', ' ', regex=True).astype(str)
ad.head()

,No voie,Type de voie,Voie,Code postal,Commune,Code departement,Code commune,Code commune INSEE,Ville,Adresse
0,NaN,NaN,FORTUNAT,1250,CEYZERIAT,01,072,01072,1250 CEYZERIAT,FORTUNAT 1250 CEYZERIAT
1,NaN,NaN,TERRES DES CINQ SAULES,1290,LAIZ,01,203,01203,1290 LAIZ,TERRES DES CINQ SAULES 1290 LAIZ
2,NaN,NaN,BOIS DU CHAMP RION,1290,LAIZ,01,203,01203,1290 LAIZ,BOIS DU CHAMP RION 1290 LAIZ
3,NaN,NaN,EN COROBERT,1290,LAIZ,01,203,01203,1290 LAIZ,EN COROBERT 1290 LAIZ
6,NaN,RUE,DE LA CHARTREUSE,1960,PERONNAS,01,289,01289,1960 PERONNAS,RUE DE LA CHARTREUSE 1960 PERONNAS


## Sélection des adresses à parser pour ajout des coordonnées GPS et codes IRIS

In [18]:
# Sélection des adresses à partir de row_first et row_last définies plus haut
selec = ad.iloc[row_first:row_last, :]
selec.shape

(500, 10)

In [19]:
# Instanciation de la BDD de correspondance entre Code Postal / Communes et Code INSEE + GPS Communes (nécessaire pour code IRIS)
# La BDD se trouve ici : https://data.opendatasoft.com/explore/dataset/code-postal-code-insee-2015@public/download/?format=csv&timezone=Europe/Berlin&lang=fr&use_labels_for_header=true&csv_separator=%3B
communes = pd.read_csv('databases/code-postal-code-insee-2015.csv', sep = ";")
communes.head()

,Geo Point,Geo Shape,CODE_COM,INSEE_COM,NOM_COM,STATUT,X_CENTROID,Y_CENTROID,Z_MOYEN,SUPERFICIE,POPULATION,CODE_CANT,CODE_ARR,CODE_DEPT,NOM_DEPT,Code_postal,Nom_commune,coordonnees_gps,Unnamed: 18
0,"49.0097923668,0.696073334129","{""type"": ""Polygon"", ""coordinates"": [[[0.682900...",49,27049,BEAUMESNIL,Commune simple,531927,6881409,162,1264,562,2.0,2.0,27,EURE,27330.0,MESNIL EN OUCHE,"49.0090520551,0.696415474297",NaN
1,"49.0521199676,0.412267826158","{""type"": ""Polygon"", ""coordinates"": [[[0.409543...",547,27547,SAINT-GERMAIN-LA-CAMPAGNE,Commune simple,510734,6886673,176,2231,895,3.0,2.0,27,EURE,27230.0,ST GERMAIN LA CAMPAGNE,"49.0522921876,0.412852228157",NaN
2,"48.7896521776,0.276557382012","{""type"": ""Polygon"", ""coordinates"": [[[0.260047...",347,61347,RESENLIEU,Commune simple,500194,6858135,231,504,210,21.0,2.0,61,ORNE,61230.0,RESENLIEU,"48.7900887821,0.276930426076",NaN
3,"48.8470682746,0.643176111978","{""type"": ""Polygon"", ""coordinates"": [[[0.628763...",359,27359,JUIGNETTES,Commune simple,527172,6863726,228,1302,243,6.0,3.0,27,EURE,27250.0,JUIGNETTES,"48.8472706788,0.64345379282",NaN
4,"48.8662500091,0.559682765078","{""type"": ""Polygon"", ""coordinates"": [[[0.553791...",136,61136,COUVAINS,Commune simple,521103,6865598,236,1788,167,18.0,2.0,61,ORNE,NaN,NaN,NaN,NaN


In [20]:
# On ne garde que les colonnes qui nous intéressent 
# (On a les coordonnees gps dedans, ce sera intéressant pour faire un check des coordonnees qu'on récupère plus bas)

communes = communes[['INSEE_COM', 'NOM_COM', 'Code_postal', 'Nom_commune', 'coordonnees_gps']]
communes = communes.rename(columns = {'NOM_COM': "Commune"})
communes.head()

,INSEE_COM,Commune,Code_postal,Nom_commune,coordonnees_gps
0,27049,BEAUMESNIL,27330.0,MESNIL EN OUCHE,"49.0090520551,0.696415474297"
1,27547,SAINT-GERMAIN-LA-CAMPAGNE,27230.0,ST GERMAIN LA CAMPAGNE,"49.0522921876,0.412852228157"
2,61347,RESENLIEU,61230.0,RESENLIEU,"48.7900887821,0.276930426076"
3,27359,JUIGNETTES,27250.0,JUIGNETTES,"48.8472706788,0.64345379282"
4,61136,COUVAINS,NaN,NaN,NaN


In [21]:
# Instanciation d'un dataframe communes_ville qui ne conserve que le code INSEE, les coordonnés, et la ville
communes_ville = communes
communes_ville['Code_postal'] = communes_ville['Code_postal'].fillna(-1).astype(int).astype(str).replace('-1', np.nan)
communes_ville['Ville'] = communes_ville['Code_postal'] + " " + communes_ville['Commune']
communes_ville['Ville'] = communes_ville['Ville'].where(-communes_ville['Code_postal'].isna(), communes_ville['Commune'])
communes_ville = communes_ville.drop(columns = {'Commune', 'Code_postal', 'Nom_commune'})
communes_ville['Ville'] = communes_ville['Ville'].replace('-', ' ', regex=True).astype(str)
communes_ville.head()

,INSEE_COM,coordonnees_gps,Ville
0,27049,"49.0090520551,0.696415474297",27330 BEAUMESNIL
1,27547,"49.0522921876,0.412852228157",27230 SAINT GERMAIN LA CAMPAGNE
2,61347,"48.7900887821,0.276930426076",61230 RESENLIEU
3,27359,"48.8472706788,0.64345379282",27250 JUIGNETTES
4,61136,NaN,COUVAINS


In [22]:
# Ajout du code commune INSEE et Coordonnées GPS à la sélection d'adresses
selec = selec.merge(communes_ville, on = "Ville", how = "left")
selec.head()

,No voie,Type de voie,Voie,Code postal,Commune,Code departement,Code commune,Code commune INSEE,Ville,Adresse,INSEE_COM,coordonnees_gps
0,123.0,AV,DE SAINT JULIEN,13012,MARSEILLE,13,212,13212,13012 MARSEILLE 12EME,123 AV DE SAINT JULIEN 13012 MARSEILLE,NaN,NaN
1,22.0,TRA,DES FAIENCIERS,13011,MARSEILLE,13,211,13211,13011 MARSEILLE 11EME,22 TRA DES FAIENCIERS 13011 MARSEILLE,NaN,NaN
2,1.0,TSSE,CAZENEUVE,13012,MARSEILLE,13,212,13212,13012 MARSEILLE 12EME,1 TSSE CAZENEUVE 13012 MARSEILLE,NaN,NaN
3,180.0,AV,DES CAILLOLS,13012,MARSEILLE,13,212,13212,13012 MARSEILLE 12EME,180 AV DES CAILLOLS 13012 MARSEILLE,NaN,NaN
4,27.0,AV,DU 24 AVRIL 1915,13012,MARSEILLE,13,212,13212,13012 MARSEILLE 12EME,27 AV DU 24 AVRIL 1915 13012 MARSEILLE,NaN,NaN


In [23]:
# Test sur Marseille car les codes communes INSEE semblent être différents selon la source
selec[(selec['Ville'].str.contains("MARSEILLE")) & (selec['Code departement'] == 13)]

,No voie,Type de voie,Voie,Code postal,Commune,Code departement,Code commune,Code commune INSEE,Ville,Adresse,INSEE_COM,coordonnees_gps
0,123.0,AV,DE SAINT JULIEN,13012,MARSEILLE,13,212,13212,13012 MARSEILLE 12EME,123 AV DE SAINT JULIEN 13012 MARSEILLE,NaN,NaN
1,22.0,TRA,DES FAIENCIERS,13011,MARSEILLE,13,211,13211,13011 MARSEILLE 11EME,22 TRA DES FAIENCIERS 13011 MARSEILLE,NaN,NaN
2,1.0,TSSE,CAZENEUVE,13012,MARSEILLE,13,212,13212,13012 MARSEILLE 12EME,1 TSSE CAZENEUVE 13012 MARSEILLE,NaN,NaN
3,180.0,AV,DES CAILLOLS,13012,MARSEILLE,13,212,13212,13012 MARSEILLE 12EME,180 AV DES CAILLOLS 13012 MARSEILLE,NaN,NaN
4,27.0,AV,DU 24 AVRIL 1915,13012,MARSEILLE,13,212,13212,13012 MARSEILLE 12EME,27 AV DU 24 AVRIL 1915 13012 MARSEILLE,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
495,25.0,BD,MERLE,13012,MARSEILLE,13,212,13212,13012 MARSEILLE 12EME,25 BD MERLE 13012 MARSEILLE,NaN,NaN
496,NaN,RES,LES CHARMILLES,13011,MARSEILLE,13,211,13211,13011 MARSEILLE 11EME,RES LES CHARMILLES 13011 MARSEILLE,NaN,NaN
497,2.0,BD,DEBORD,13012,MARSEILLE,13,212,13212,13012 MARSEILLE 12EME,2 BD DEBORD 13012 MARSEILLE,NaN,NaN
498,255.0,AV,DE MONTOLIVET,13012,MARSEILLE,13,212,13212,13012 MARSEILLE 12EME,255 AV DE MONTOLIVET 13012 MARSEILLE,NaN,NaN


In [24]:
selec['Commune_Lat'] = selec['coordonnees_gps'].str.split(",", expand = True)[0]
selec['Commune_Lon'] = selec['coordonnees_gps'].str.split(",", expand = True)[1]
selec = selec.drop(columns = {'coordonnees_gps'})
selec.head(15)

,No voie,Type de voie,Voie,Code postal,Commune,Code departement,Code commune,Code commune INSEE,Ville,Adresse,INSEE_COM,Commune_Lat,Commune_Lon
0,123.0,AV,DE SAINT JULIEN,13012,MARSEILLE,13,212,13212,13012 MARSEILLE 12EME,123 AV DE SAINT JULIEN 13012 MARSEILLE,NaN,NaN,NaN
1,22.0,TRA,DES FAIENCIERS,13011,MARSEILLE,13,211,13211,13011 MARSEILLE 11EME,22 TRA DES FAIENCIERS 13011 MARSEILLE,NaN,NaN,NaN
2,1.0,TSSE,CAZENEUVE,13012,MARSEILLE,13,212,13212,13012 MARSEILLE 12EME,1 TSSE CAZENEUVE 13012 MARSEILLE,NaN,NaN,NaN
3,180.0,AV,DES CAILLOLS,13012,MARSEILLE,13,212,13212,13012 MARSEILLE 12EME,180 AV DES CAILLOLS 13012 MARSEILLE,NaN,NaN,NaN
4,27.0,AV,DU 24 AVRIL 1915,13012,MARSEILLE,13,212,13212,13012 MARSEILLE 12EME,27 AV DU 24 AVRIL 1915 13012 MARSEILLE,NaN,NaN,NaN
5,7.0,RUE,DE VALMY,13012,MARSEILLE,13,212,13212,13012 MARSEILLE 12EME,7 RUE DE VALMY 13012 MARSEILLE,NaN,NaN,NaN
6,20.0,TRA,DES FAIENCIERS,13011,MARSEILLE,13,211,13211,13011 MARSEILLE 11EME,20 TRA DES FAIENCIERS 13011 MARSEILLE,NaN,NaN,NaN
7,47.0,AV,DES TROIS LUCS,13012,MARSEILLE,13,212,13212,13012 MARSEILLE 12EME,47 AV DES TROIS LUCS 13012 MARSEILLE,NaN,NaN,NaN
8,93.0,BD,HENRI FABRE,13012,MARSEILLE,13,212,13212,13012 MARSEILLE 12EME,93 BD HENRI FABRE 13012 MARSEILLE,NaN,NaN,NaN
9,100.0,CHE,DU COLLET BLANC,13190,ALLAUCH,13,002,13002,13190 ALLAUCH,100 CHE DU COLLET BLANC 13190 ALLAUCH,13002,43.3533145712,5.51134767187


In [25]:
selec['code_iris'] = ""
selec['lon'] = ""
selec['lat'] = ""
selec.head()

,No voie,Type de voie,Voie,Code postal,Commune,Code departement,Code commune,Code commune INSEE,Ville,Adresse,INSEE_COM,Commune_Lat,Commune_Lon,code_iris,lon,lat
0,123.0,AV,DE SAINT JULIEN,13012,MARSEILLE,13,212,13212,13012 MARSEILLE 12EME,123 AV DE SAINT JULIEN 13012 MARSEILLE,NaN,NaN,NaN,,,
1,22.0,TRA,DES FAIENCIERS,13011,MARSEILLE,13,211,13211,13011 MARSEILLE 11EME,22 TRA DES FAIENCIERS 13011 MARSEILLE,NaN,NaN,NaN,,,
2,1.0,TSSE,CAZENEUVE,13012,MARSEILLE,13,212,13212,13012 MARSEILLE 12EME,1 TSSE CAZENEUVE 13012 MARSEILLE,NaN,NaN,NaN,,,
3,180.0,AV,DES CAILLOLS,13012,MARSEILLE,13,212,13212,13012 MARSEILLE 12EME,180 AV DES CAILLOLS 13012 MARSEILLE,NaN,NaN,NaN,,,
4,27.0,AV,DU 24 AVRIL 1915,13012,MARSEILLE,13,212,13212,13012 MARSEILLE 12EME,27 AV DU 24 AVRIL 1915 13012 MARSEILLE,NaN,NaN,NaN,,,


In [26]:
selec.shape

(500, 16)

## Récupération des codes IRIS et coordonnées GPS à partir de l'API Nominatim et geo.api.gouv

In [28]:
# Get GPS Coordinates + IRIS Code from addresses

geolocator = Nominatim(user_agent="pyPredImmo")

start = time.time()
etape = time.time()
print("Heure début : ")
print(time.strftime('%H:%M:%S', time.gmtime(start)))
print("\n")
n=0

for row in range(0, row_last - row_first):
    
    iris = ""
    longitude_api = ""
    latitude_api = ""
    longitude_adresse = ""
    latitude_adresse = ""

    
    adresse = selec['Adresse'][row]
    ville = selec['Ville'][row]
    code_INSEE = selec['Code commune INSEE'][row]

    # Interrogation de l'API Nominatim
    try:
        location = geolocator.geocode(adresse, country_codes = 'fr', timeout = 5)
    
    except requests.exceptions.RequestException as e:
        latitude_api = ""
        longitude_api = ""
        raise SystemExit(e)
        
    if location is None:                      # Sinon aucun résultat, on ne met rien dans les coordonnées GPS qui vont servir à interroger les codes IRIS
        latitude_api = ""
        longitude_api = ""
    
    else:
        latitude_api = location.latitude
        longitude_api = location.longitude
        
        if isinstance(code_INSEE, float):     # Si pas de code commune
            print(adresse + ': pas de code commune INSEE')
            iris = ""
        elif latitude_api == "":               # Si pas de coordonnées GPS
            print(adresse + ": pas de coordonnees GPS pour code IRIS")
            iris = ""
        else:                              # Sinon on interroge l'API du gouv pour récupérer le code IRIS
            try:
                result = requests.get('https://geo.api.gouv.fr/iris?lon=' + str(longitude_api) + '&lat=' 
                                 + str(latitude_api) + '&codeCommune=' + str(code_INSEE), timeout = 5)
            except requests.exceptions.RequestException as e: 
                raise SystemExit(e)
            
            if result.ok == True:
                obj = json.loads(result.text)
                iris = obj['codeIris']
                #print(obj['codeIris'])
            else:
                result.close()
            print(adresse + " ===> " + str(latitude_api) + "," + str(longitude_api) + " (code IRIS = " + str(iris) + ")")
        
    selec['code_iris'][row] = iris
    selec['lon'][row] = longitude_api
    selec['lat'][row] = latitude_api
    
    n = n + 1
    
    if (n % 100 == 0):
        print("\n")
        print("Etape : " + str(n))
        print(time.strftime('%H:%M:%S', time.gmtime(time.time())))
        print("Temps pour 100 : " + str(time.time() - etape) + " sec")
        print("\n")
        etape = time.time()

    if (n % 10000 == 0):
        selec.iloc[(n - 10000) : n, :].to_csv(
            'export_adresses_inter_' + str(row_first + n - 10000) + "-" + str(row_first + n)+'.csv')
        
end = time.time()
print("\n")
print("Heure fin : ")
print(time.strftime('%H:%M:%S', time.gmtime(end)))
print("\n")

print("Temps de calcul : ")
print(str(end - start) + ' sec')

Heure début : 
13:51:17


123 AV DE SAINT JULIEN 13012 MARSEILLE ===> 43.3075361,5.4253692 (code IRIS = 132120401)
180 AV DES CAILLOLS 13012 MARSEILLE ===> 43.3040841,5.4253068 (code IRIS = 132120201)
27 AV DU 24 AVRIL 1915 13012 MARSEILLE ===> 43.3076019,5.4256536 (code IRIS = 132120603)
7 RUE DE VALMY 13012 MARSEILLE ===> 43.3120186,5.4190449 (code IRIS = 132120401)
47 AV DES TROIS LUCS 13012 MARSEILLE ===> 43.311070400000006,5.461810805434448 (code IRIS = 132120702)
93 BD HENRI FABRE 13012 MARSEILLE ===> 43.3080342,5.4162837 (code IRIS = 132120402)
100 CHE DU COLLET BLANC 13190 ALLAUCH ===> 43.3329466,5.4750976 (code IRIS = 130020105)
8 AV DES BORROMEES 13012 MARSEILLE ===> 43.3014274,5.4313173 (code IRIS = 132120202)
137 RUE ALPHONSE DAUDET 13013 MARSEILLE ===> 43.322268,5.4126024 (code IRIS = 132130304)
181 RUE DU DOCTEUR CAUVIN 13012 MARSEILLE ===> 43.2981107,5.4176132 (code IRIS = 132120501)
18 RUE CHALUSSET 13013 MARSEILLE ===> 43.3168559,5.3991878 (code IRIS = 132131004)
194 A

22 AV DU 24 AVRIL 1915 13012 MARSEILLE ===> 43.3076019,5.4256536 (code IRIS = 132120603)
34 BD GILLET 13012 MARSEILLE ===> 43.3123901,5.4156157 (code IRIS = 132120302)
101 RUE ALPHONSE DAUDET 13013 MARSEILLE ===> 43.322268,5.4126024 (code IRIS = 132130304)
60 BD PINATEL 13012 MARSEILLE ===> 43.3146072,5.4414064 (code IRIS = 132120604)
CHE DES SABLES JAUNES 13012 MARSEILLE ===> 43.3125518,5.4306865 (code IRIS = 132120603)
65 RUE DE L'AIGUILLETTE 13012 MARSEILLE ===> 43.311545,5.4282117 (code IRIS = 132120401)
12 RUE DE L'ESCALET 13013 MARSEILLE ===> 43.352616,5.4159269 (code IRIS = 132130501)
40 BD BOUYALA D'ARNAUD 13012 MARSEILLE ===> 43.2996417,5.4397956 (code IRIS = 132120103)
19 RUE DES BONS AMIS 13012 MARSEILLE ===> 43.2970374,5.4172407 (code IRIS = 132120501)
123 RUE ALPHONSE DAUDET 13013 MARSEILLE ===> 43.322268,5.4126024 (code IRIS = 132130304)
2 RUE DE LA SAOUPE 13011 MARSEILLE ===> 43.2935903,5.5017031 (code IRIS = 132110801)
142 RUE ALPHONSE DAUDET 13013 MARSEILLE ===> 43.322

195 CHE DES RASCOUS 13190 ALLAUCH ===> 43.3451745,5.4765598 (code IRIS = 130020103)
5525 CHE DES RASCOUS 13190 ALLAUCH ===> 43.3451745,5.4765598 (code IRIS = 130020103)
88 BD DES FAUVETTES 13012 MARSEILLE ===> 43.3148638,5.4391167 (code IRIS = 132120604)
283 AV DES POILUS 13012 MARSEILLE ===> 43.3171812,5.4604861 (code IRIS = 132120701)
466 CHE DE LA CRAIE 13190 ALLAUCH ===> 43.3293044,5.4904175 (code IRIS = 130020106)
174 BD DE LA BARASSE 13011 MARSEILLE ===> 43.2859559,5.4795967 (code IRIS = 132110201)
28 BD DES LIBERATEURS 13011 MARSEILLE ===> 43.2906199,5.4632152 (code IRIS = 132110704)
9 RUE ETIENNE PARROCEL 13013 MARSEILLE ===> 43.3181861,5.3989886 (code IRIS = 132131002)
378 CHE LOUIS DELESTRADE 13190 ALLAUCH ===> 43.3250504,5.4860341 (code IRIS = 130020106)
22 AV DES PLATANES 13012 MARSEILLE ===> 43.3128748,5.4309303 (code IRIS = 132120603)
72 IMP DE LA CHAPELLE 13013 MARSEILLE ===> 43.3323461,5.4263926 (code IRIS = 132130804)
82 BD APOLLON 13011 MARSEILLE ===> 43.2817708,5.492

58 AV DES CAILLOLS 13012 MARSEILLE ===> 43.3040841,5.4253068 (code IRIS = 132120201)
11 RUE BAILLI DE SUFFREN 13380 PLAN DE CUQUES ===> 43.3547487,5.4664347 (code IRIS = 130750105)
3 CHE DU CIMETIERE 13012 MARSEILLE ===> 43.3038919,5.4521342 (code IRIS = 132120101)
35 IMP DES DOUCES 13011 MARSEILLE ===> 43.3107062,5.5044245 (code IRIS = 132110901)
21 CHE DE CHATEAU GOMBERT 13013 MARSEILLE ===> 43.3514323,5.433358 (code IRIS = 132131103)
3 BD BERNARD VERGER 13013 MARSEILLE ===> 43.3595377,5.4267897 (code IRIS = 132130501)
25  LE CIMETIERE 13011 MARSEILLE ===> 43.3029426,5.4939061 (code IRIS = 132110101)
LE CIMETIERE 13011 MARSEILLE ===> 43.3029426,5.4939061 (code IRIS = 132110101)
49 AV LEI RIMA 13190 ALLAUCH ===> 43.3572007,5.4885237 (code IRIS = 130020101)
38 TRA TRIVIER 13012 MARSEILLE ===> 43.3118251,5.4075673 (code IRIS = 132120301)
5 RUE AUGUSTIN FRESNEL 13013 MARSEILLE ===> 43.3428868,5.4473116 (code IRIS = 132130102)
1577  LA MONTADE 13380 PLAN DE CUQUES ===> 43.36059365,5.46579

In [29]:
selec.head()

,No voie,Type de voie,Voie,Code postal,Commune,Code departement,Code commune,Code commune INSEE,Ville,Adresse,INSEE_COM,Commune_Lat,Commune_Lon,code_iris,lon,lat
0,123.0,AV,DE SAINT JULIEN,13012,MARSEILLE,13,212,13212,13012 MARSEILLE 12EME,123 AV DE SAINT JULIEN 13012 MARSEILLE,NaN,NaN,NaN,132120401,5.425369,43.307536
1,22.0,TRA,DES FAIENCIERS,13011,MARSEILLE,13,211,13211,13011 MARSEILLE 11EME,22 TRA DES FAIENCIERS 13011 MARSEILLE,NaN,NaN,NaN,,,
2,1.0,TSSE,CAZENEUVE,13012,MARSEILLE,13,212,13212,13012 MARSEILLE 12EME,1 TSSE CAZENEUVE 13012 MARSEILLE,NaN,NaN,NaN,,,
3,180.0,AV,DES CAILLOLS,13012,MARSEILLE,13,212,13212,13012 MARSEILLE 12EME,180 AV DES CAILLOLS 13012 MARSEILLE,NaN,NaN,NaN,132120201,5.425307,43.304084
4,27.0,AV,DU 24 AVRIL 1915,13012,MARSEILLE,13,212,13212,13012 MARSEILLE 12EME,27 AV DU 24 AVRIL 1915 13012 MARSEILLE,NaN,NaN,NaN,132120603,5.425654,43.307602


In [30]:
selec.shape

(500, 16)

## Export de la totalité de la sélection d'adresses après la fin du job

In [31]:
selec.to_csv('export_adresses_' + str(row_first) + "-" + str(row_last)+'.csv')